In [36]:
import h5py
import numpy as np
import rasterio
import os

bin_dir = os.getcwd()
pkg_dir = os.path.dirname(bin_dir)
data_dir = os.path.join(pkg_dir, 'tests', 'data', 'ri_exclusions')

In [46]:
path = os.path.join(data_dir, 'exclusions_padus.tif')
with rasterio.open(path, 'r') as file:
    print(file.shape)
    print(file.indexes)
    print(file.profile)
    data = np.array([file.read(i) for i in file.indexes])
    print(file.name)

(1434, 972)
(1,)
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 972, 'height': 1434, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["GRS 1980(IUGG, 1980)",DATUM["unknown",SPHEROID["GRS80",6378137,298.257222101],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",41.08333333333334],PARAMETER["central_meridian",-71.5],PARAMETER["scale_factor",0.99999375],PARAMETER["false_easting",100000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'), 'transform': Affine(90.0, 0.0, 65848.61752782026,
       0.0, -90.0, 103948.14381277534), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
/Users/mrossol/Git_Repos/reV/tests/data/ri_exclusions/exclusions_padus.tif


In [51]:
with xr.open_rasterio(path, chunks=(128, 128)) as f:
    print(f.attrs['transform'])

(90.0, 0.0, 65848.61752782026, 0.0, -90.0, 103948.14381277534)
